# Stochastic Variant Read Simulation with Binomial Model

### Alexander Munoz, Margaux Hujoel, Emily Simons

In [2]:
import numpy as np
import pandas as pd
import math

In [37]:
##### SIMULATOR IMPLEMENTATION #####

class Simulator(object):
    ''' N is number of loci, M is number of samples, D_locus is average read depth per locus, 
        c_range is possible number of clones to test '''
    
    def __init__(self, loci_num, samples_num, locus_read_depth, clone_test_vals=[3,4,5]):
        self.N = loci_num
        self.M = samples_num
        self.D_locus = locus_read_depth
        self.c_range = clone_test_vals
        
    def simulate_R(self, print_bool=False):
        simulated_R = np.zeros((self.N,self.M))
        for col_num in range(self.M):
            simulated_R[:,col_num] = np.random.multinomial(self.D_locus * self.N, [1/self.N]*self.N)
        if print_bool:
            print(simulated_R)
        return simulated_R
    
    def simulate_Zs(self, p, print_bool=False):
        simulated_Zs = [np.zeros((self.N, c)) for c in self.c_range]
        for z_index in range(len(simulated_Zs)):
            simulated_z = simulated_Zs[z_index]
            for col_idx in range(simulated_z.shape[1]-1):
                simulated_Zs[z_index][:,col_idx+1] = [np.random.binomial(1,p) for j in range(simulated_z.shape[0])]
        if print_bool:
            print(simulated_Zs)
        return simulated_Zs
    
    def simulate_Ps(self, print_bool=False):
        simulated_Ps = [np.zeros((c, self.M)) for c in self.c_range]
        for p_index in range(len(simulated_Ps)):
            simulated_Ps[p_index][0,0] = 1
            simulated_p = simulated_Ps[p_index] 
            for col_idx in range(simulated_p.shape[1]-1):
                curr_col = np.array([np.random.uniform() for j in range(simulated_p.shape[0])])
                simulated_Ps[p_index][:,col_idx+1] = curr_col / np.sum(curr_col)
        if print_bool:
            print(simulated_Ps)
        return simulated_Ps
    
    def simulate_Xs_R (self, p=0.7, print_bool=False):
        simulated_R = self.simulate_R()
        simulated_Zs = self.simulate_Zs(p=p)
        simulated_Ps = self.simulate_Ps()
        simulated_Xs = [np.zeros((self.N, self.M)) for c in self.c_range]
        for x_index in range(len(simulated_Xs)):
            for x_row in range(self.N):
                for x_col in range(self.M):
                    simulated_Xs[x_index][x_row, x_col] = \
                    np.random.binomial(simulated_R[x_row, x_col], \
                                       0.5*np.dot(simulated_Zs[x_index][x_row,:], \
                                                  simulated_Ps[x_index][:, x_col]))
        if print_bool:
            print(simulated_Xs)
        return (simulated_Xs, simulated_R)
    
    def simulate_Vs (self, simulated_Xs, simulated_R, print_bool=False):
        #let V be the percent variant reads at each locus and sample, [N x M] matrix
        simulated_Vs = []
        for i in range(len(simulated_Xs)):
            simulated_V_curr = simulated_Xs[i] / simulated_R
            simulated_Vs.append(simulated_V_curr)
        if print_bool:
            for idx in range(len(simulated_Vs)):
                print("Testing with %i clones." % self.c_range[idx])
                print("V shape: ", simulated_Vs[idx].shape)
                print(simulated_Vs[idx])
                print()
        return simulated_Vs

In [38]:
##### INITIALIZE HYPERPARAMETERS #####

loci_num = 50
samples_num = 10
locus_read_depth = 25
clone_test_vals = [3,4,5]
p = 0.7

In [39]:
##### SIMULATE DATA #####

simulator = Simulator(loci_num=loci_num, \
                      samples_num=samples_num, \
                      locus_read_depth=locus_read_depth, \
                      clone_test_vals=clone_test_vals)

simulated_Xs, simulated_R = simulator.simulate_Xs_R(p=p)
simulated_Vs = simulator.simulate_Vs(simulated_Xs=simulated_Xs, simulated_R=simulated_R, print_bool=True)

Testing with 3 clones.
V shape:  (50, 10)
[[ 0.          0.21428571  0.03448276  0.05263158  0.13636364  0.16129032
   0.25806452  0.25925926  0.          0.4       ]
 [ 0.          0.23529412  0.14285714  0.14814815  0.38461538  0.17241379
   0.46666667  0.43478261  0.03703704  0.2962963 ]
 [ 0.          0.4         0.18181818  0.32        0.36        0.15384615
   0.41666667  0.42857143  0.06451613  0.37037037]
 [ 0.          0.55172414  0.05555556  0.25        0.46428571  0.27777778
   0.35714286  0.28571429  0.08695652  0.25806452]
 [ 0.          0.34482759  0.1         0.30434783  0.5862069   0.36842105
   0.59090909  0.48148148  0.05263158  0.29166667]
 [ 0.          0.33333333  0.08695652  0.18181818  0.6         0.08        0.6
   0.72222222  0.2         0.16666667]
 [ 0.          0.375       0.0952381   0.28        0.34615385  0.2         0.5
   0.47826087  0.          0.34482759]
 [ 0.          0.13636364  0.          0.08        0.09375     0.38461538
   0.2173913   0.115384